In [34]:
import re
import pandas as pd

from collections import Counter


In [35]:
with open('../data/ViMedNER.txt', 'r', encoding='utf-8') as f:
    noi_dung = f.read()

In [36]:
import json


dataset = []
with open('../data/ViMedNER.txt', 'r', encoding='utf-8') as f:
    for dong in f:
        dataset.append(json.loads(dong.strip())) # .strip() để loại bỏ ký tự xuống dòng (\n) thừa

In [37]:
def count_keys(dataset):
    keys = [key for d in dataset for key in d.keys()]

    counter = Counter(keys)
    return counter

In [38]:
keys_count = count_keys(dataset)
print(keys_count)

Counter({'id': 7749, 'text': 7749, 'label': 7749, 'Comments': 7749})


In [39]:
df = pd.DataFrame(dataset)
df.head()

,id,text,label,Comments
0,3504,kháng sinh dự phòng nếu đã được chỉ định trong...,"[[135, 163, DISEASE]]",[]
1,3505,ngày nay các chuyên gia tin rằng viêm nội tâm ...,"[[33, 49, DISEASE], [101, 109, CAUSE]]",[]
2,3506,hướng dẫn hiện nay khuyến cáo điều trị kháng s...,"[[129, 157, DISEASE]]",[]
3,3507,bác sĩ vẫn có thể khuyên nên dùng kháng sinh p...,"[[84, 100, TREATMENT]]",[]
4,3508,đối với hầu hết những người có thông liên thất...,"[[31, 46, DISEASE], [160, 176, DISEASE]]",[]


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7749 entries, 0 to 7748
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7749 non-null   int64 
 1   text      7749 non-null   object
 2   label     7749 non-null   object
 3   Comments  7749 non-null   object
dtypes: int64(1), object(3)
memory usage: 242.3+ KB


In [41]:
print(df['Comments'].value_counts())
df = df.drop(columns='Comments')

Comments
[]    7749
Name: count, dtype: int64


In [42]:
df_exploded = df['label'].explode()
label_counts = df_exploded.dropna().apply(lambda x: x[2]).value_counts()
print(label_counts)

label
DISEASE       9712
SYMPTOM       4195
TREATMENT     3084
DIAGNOSTIC    1559
CAUSE         1433
Name: count, dtype: int64


In [43]:
def get_entity_spans(label_list):
    return set([item[2] for item in label_list])

In [44]:
import itertools
co_occurrences = []
for labels in df['label']:
    types = get_entity_spans(labels)
    if len(types) > 1:
        pairs = list(itertools.combinations(sorted(list(types)), 2))
        co_occurrences.extend(pairs)
        
pair_counts = Counter(co_occurrences)
for pair, count in pair_counts.most_common():
    print(f"{pair[0]} - {pair[1]}: {count} times")

DISEASE - SYMPTOM: 921 times
DISEASE - TREATMENT: 748 times
CAUSE - DISEASE: 560 times
DIAGNOSTIC - DISEASE: 393 times
SYMPTOM - TREATMENT: 191 times
CAUSE - SYMPTOM: 127 times
DIAGNOSTIC - SYMPTOM: 82 times
CAUSE - TREATMENT: 80 times
DIAGNOSTIC - TREATMENT: 72 times
CAUSE - DIAGNOSTIC: 37 times


In [45]:
import itertools
from collections import Counter

TARGET_RELATIONS = {
    frozenset(['DISEASE', 'SYMPTOM']),
    frozenset(['DISEASE', 'TREATMENT']),
    frozenset(['CAUSE', 'DISEASE']),
    frozenset(['DIAGNOSTIC', 'DISEASE'])
}

def count_relations_with_ratio(data, ratio=4):
    positive_counter = Counter()
    negative_counter = Counter()
    
    total_pos = 0
    total_neg = 0

    for entry in data:

        entity_types = set([item[2] for item in entry['label']])
        
        if len(entity_types) < 2:
            continue

        # Tạo tất cả các cặp có thể trong câu
        # Sử dụng combinations để tạo cặp không lặp lại vị trí
        pairs = list(itertools.combinations(entity_types, 2))
        
        # Danh sách tạm cho từng câu
        sent_positives = []
        sent_negatives = []

        # Phân loại cặp
        for p in pairs:
            # Chuyển pair thành set để so sánh với TARGET_RELATIONS
            p_set = frozenset(p)
            
            # Tạo chuỗi đại diện để đếm (Sắp xếp A-Z để thống nhất hiển thị)
            # Ví dụ: Luôn hiển thị "DISEASE - SYMPTOM" thay vì lúc thì "SYMPTOM - DISEASE"
            p_str = f"{sorted(p)[0]} - {sorted(p)[1]}"

            if p_set in TARGET_RELATIONS:
                sent_positives.append(p_str)
            else:
                sent_negatives.append(p_str)

        # --- XỬ LÝ TỶ LỆ 1:4 (Logic quan trọng) ---
        num_pos = len(sent_positives)
        
        # Giới hạn số lượng No-Relation: gấp 4 lần số Positive
        # Lưu ý: Nếu num_pos = 0 thì max_neg = 0 (theo yêu cầu chặt chẽ của bạn)
        max_neg = num_pos * ratio
        
        # Cắt list negative lấy đúng số lượng cho phép
        # (Ở đây lấy theo thứ tự xuất hiện, nếu muốn ngẫu nhiên thì thêm random.shuffle)
        valid_negatives = sent_negatives[:max_neg]

        # --- CẬP NHẬT COUNT TỔNG ---
        positive_counter.update(sent_positives)
        negative_counter.update(valid_negatives)
        
        total_pos += num_pos
        total_neg += len(valid_negatives)

    return positive_counter, negative_counter, total_pos, total_neg

pos_counts, neg_counts, t_pos, t_neg = count_relations_with_ratio(dataset)

print("=== KẾT QUẢ ĐẾM (POSITIVE RELATIONS) ===")
for pair, count in pos_counts.most_common():
    print(f"{pair}: {count}")
print(f"-> Tổng cộng Positive: {t_pos}")

print("\n=== KẾT QUẢ ĐẾM (NO-RELATION / NEGATIVE) - Đã lọc theo tỷ lệ 1:4 ===")
if not neg_counts:
    print("Không có cặp No-Relation nào được chọn (do không đủ Positive để kéo theo).")
else:
    for pair, count in neg_counts.most_common():
        print(f"{pair}: {count}")
print(f"-> Tổng cộng No-Relation: {t_neg}")

=== KẾT QUẢ ĐẾM (POSITIVE RELATIONS) ===
DISEASE - SYMPTOM: 921
DISEASE - TREATMENT: 748
CAUSE - DISEASE: 560
DIAGNOSTIC - DISEASE: 393
-> Tổng cộng Positive: 2622

=== KẾT QUẢ ĐẾM (NO-RELATION / NEGATIVE) - Đã lọc theo tỷ lệ 1:4 ===
SYMPTOM - TREATMENT: 85
CAUSE - SYMPTOM: 64
CAUSE - TREATMENT: 57
DIAGNOSTIC - SYMPTOM: 50
DIAGNOSTIC - TREATMENT: 39
CAUSE - DIAGNOSTIC: 18
-> Tổng cộng No-Relation: 313


In [49]:
import itertools
from collections import Counter

# --- Cấu hình các cặp Positive Relation ---
TARGET_RELATIONS = {
    frozenset(['DISEASE', 'SYMPTOM']),
    frozenset(['DISEASE', 'TREATMENT']),
    frozenset(['CAUSE', 'DISEASE']),
    frozenset(['DIAGNOSTIC', 'DISEASE'])
}

def count_relations_with_ratio(data, ratio=100):
    positive_counter = Counter()
    negative_counter = Counter()
    
    total_pos = 0
    total_neg = 0

    for entry in data:
        # LƯU Ý QUAN TRỌNG: 
        # Bạn nên dùng list thay vì set. Nếu dùng set, các entity cùng loại (VD: 2 bệnh trong 1 câu) 
        # sẽ bị gộp thành 1, dẫn đến mất cặp Negative "DISEASE - DISEASE".
        entity_types = [item[2] for item in entry['label']]
        
        if len(entity_types) < 2:
            continue

        # Tạo tất cả các cặp
        pairs = list(itertools.combinations(entity_types, 2))
        
        sent_positives = []
        sent_negatives = []

        # Phân loại
        for p in pairs:
            p_set = frozenset(p)
            p_str = f"{sorted(p)[0]} - {sorted(p)[1]}"

            if p_set in TARGET_RELATIONS:
                sent_positives.append(p_str)
            else:
                sent_negatives.append(p_str)

        # --- [ĐÃ SỬA] XỬ LÝ LOGIC LẤY MẪU ---
        num_pos = len(sent_positives)
        
        if num_pos == 0:
            # TRƯỜNG HỢP 1: Không có Positive nào -> Lấy TẤT CẢ Negative tìm được
            valid_negatives = sent_negatives
        else:
            max_neg = num_pos * ratio
            valid_negatives = sent_negatives[:max_neg]

        # --- Cập nhật Count ---
        positive_counter.update(sent_positives)
        negative_counter.update(valid_negatives)
        
        total_pos += num_pos
        total_neg += len(valid_negatives)

    return positive_counter, negative_counter, total_pos, total_neg

pos_counts, neg_counts, t_pos, t_neg = count_relations_with_ratio(dataset)

print("=== KẾT QUẢ ĐẾM (POSITIVE RELATIONS) ===")
for pair, count in pos_counts.most_common():
    print(f"{pair}: {count}")
print(f"-> Tổng cộng Positive: {t_pos}")

print("\n=== KẾT QUẢ ĐẾM (NO-RELATION / NEGATIVE) ===")
if not neg_counts:
    print("Không có cặp No-Relation nào được chọn.")
else:
    for pair, count in neg_counts.most_common():
        print(f"{pair}: {count}")
print(f"-> Tổng cộng No-Relation: {t_neg}")

=== KẾT QUẢ ĐẾM (POSITIVE RELATIONS) ===
DISEASE - SYMPTOM: 6162
DISEASE - TREATMENT: 2418
DIAGNOSTIC - DISEASE: 2029
CAUSE - DISEASE: 1926
-> Tổng cộng Positive: 12535

=== KẾT QUẢ ĐẾM (NO-RELATION / NEGATIVE) ===
DISEASE - DISEASE: 10640
SYMPTOM - SYMPTOM: 7614
TREATMENT - TREATMENT: 3116
DIAGNOSTIC - DIAGNOSTIC: 1493
CAUSE - CAUSE: 1277
SYMPTOM - TREATMENT: 754
DIAGNOSTIC - SYMPTOM: 547
CAUSE - SYMPTOM: 527
DIAGNOSTIC - TREATMENT: 238
CAUSE - TREATMENT: 180
CAUSE - DIAGNOSTIC: 68
-> Tổng cộng No-Relation: 26454


In [47]:
same_type_co_occurrences = []

for entry in dataset:
    types = [item[2] for item in entry['label']]

    if len(types) > 1:
        pairs = list(itertools.combinations(sorted(types), 2))
        
        for p1, p2 in pairs:
            if p1 == p2:
                same_type_co_occurrences.append(f"{p1} - {p2}")

pair_counts = Counter(same_type_co_occurrences)

print("Số lượng các cặp đồng dạng:")
if not pair_counts:
    print("Không tìm thấy cặp đồng dạng nào.")
else:
    for pair_str, count in pair_counts.most_common():
        print(f"{pair_str}: {count} lần")

Số lượng các cặp đồng dạng:
DISEASE - DISEASE: 10640 lần
SYMPTOM - SYMPTOM: 7614 lần
TREATMENT - TREATMENT: 3116 lần
DIAGNOSTIC - DIAGNOSTIC: 1493 lần
CAUSE - CAUSE: 1277 lần
